# 1. Charger les données et importer des bibliothèques

In [2]:
import pandas as pd
films = pd.read_csv('export_films.csv', sep="\t", low_memory=False)

Il existe des films avec le même titre. Cela peut poser des problèmes pour le machine learning. Je vois 2 solutions
* faire plus de filtre pour enlever les doublons
* au moment de la saisie du titre par l'utilisateur on lui propose de choisir le film (cela lui propose l'année, le directeur et le genre des films ayant le même titre)

In [27]:
count_title = films['title'].value_counts()

In [28]:
count_title[count_title.values>1]

,count
title,
Hero,5
On the Edge,4
The Killer,3
The Island,3
Selfie,3
...,...
Boss,2
A Perfect Plan,2
The End,2


In [29]:
df = pd.merge(films, count_title, on='title', how='left')
df

,Unnamed: 0.1,Unnamed: 0,adult,budget,genres_x,id_tmdb,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,title,vote_average,vote_count,directors,writers,year,count
0,0,70,False,12800000,"['Drama', 'Crime']",16,tt0168629,en,Dancer in the Dark,"Selma, a Czech immigrant on the verge of blind...",...,141,['en'],Released,Dancer in the Dark,7.881,1555,nm0001885,"nm0001885,nm0797604",2000,1
1,1,109,False,140000000,"['Adventure', 'Fantasy', 'Action']",22,tt0325980,en,Pirates of the Caribbean: The Curse of the Bla...,"Jack Sparrow, a freewheeling 18th-century pira...",...,143,['en'],Released,Pirates of the Caribbean: The Curse of the Bla...,7.800,18710,nm0893659,"nm0254645,nm0744429,nm0064181,nm0938684",2003,1
2,2,133,False,30000000,"['Action', 'Crime']",24,tt0266697,en,Kill Bill: Vol. 1,"An assassin is shot by her ruthless employer, ...",...,111,"['en', 'ja', 'fr']",Released,Kill Bill: Vol. 1,7.970,15748,nm0000233,"nm0000233,nm0000235",2003,1
3,3,154,False,200000000,"['Adventure', 'Fantasy', 'Action']",58,tt0383574,en,Pirates of the Caribbean: Dead Man's Chest,Captain Jack Sparrow works his way out of a bl...,...,151,"['en', 'tr', 'el', 'zh']",Released,Pirates of the Caribbean: Dead Man's Chest,7.340,14465,nm0893659,"nm0254645,nm0744429,nm0064181,nm0938684",2006,1
4,4,196,False,2000000,"['Thriller', 'Crime', 'Drama']",67,tt0445620,en,Paradise Now,Two childhood friends are recruited for a suic...,...,90,"['ar', 'en']",Released,Paradise Now,6.941,219,nm0009463,"nm0009463,nm1178208,nm0388293",2005,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11011,12765,767462,False,0,['Drama'],215034,tt3118628,en,Bastardo,Larger-than-life characters populate director ...,...,106,['ar'],Released,Bastard,7.000,1,nm0067946,"nm0067946,nm2144991",2013,1
11012,12767,768121,False,0,['Drama'],219173,tt1576452,es,Por tu culpa,"Are we what others see, or are we what we allo...",...,87,['es'],Released,It's Your Fault,6.000,5,nm1318393,"nm1318393,nm1196938",2010,1
11013,12768,768702,False,0,['Action'],222409,tt1512732,en,Braille,Nick's blindness is symbolic to his regret for...,...,90,[],Released,Braille,6.000,1,nm2065393,"nm2065393,nm1863128",2011,1
11014,12769,769648,False,0,['Documentary'],228655,tt2007418,fr,Les trois disparitions de Soad Hosni,This haunting and beautifully formed documenta...,...,70,"['ar', 'en', 'fr']",Released,The Three Disappearances of Soad Hosni,6.000,1,nm1645751,nm1645751,2011,1


In [30]:
df[df['count']>1]

,Unnamed: 0.1,Unnamed: 0,adult,budget,genres_x,id_tmdb,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,title,vote_average,vote_count,directors,writers,year,count
6,6,237,False,31000000,"['Drama', 'Adventure', 'Action', 'History']",79,tt0299977,zh,英雄,One man defeated three assassins who sought to...,...,99,['zh'],Released,Hero,7.507,1950,nm0955443,"nm1104077,nm0955443,nm0910841",2002,5
50,50,2713,False,46000000,"['Action', 'Adventure', 'Crime', 'Thriller']",479,tt0162650,en,Shaft,New York police detective John Shaft arrests W...,...,99,"['en', 'es', 'de']",Released,Shaft,5.977,1115,nm0005436,"nm0862781,nm0005436,nm0004307,nm0697115",2000,2
74,74,3920,False,3000000,"['Drama', 'Thriller', 'Mystery', 'Action']",670,tt0364569,ko,올드보이,"With no clue how he came to be imprisoned, dru...",...,120,['ko'],Released,Oldboy,8.268,7444,nm0661791,"nm1628380,nm2057405,nm0661791,nm1367409,nm1369364",2003,2
111,111,6194,False,25000000,"['Drama', 'Action', 'Thriller', 'Crime']",1441,tt0445946,en,The Contract,Attempting to recover from a recent family tra...,...,96,['en'],Released,The Contract,5.717,431,nm0000915,"nm0441859,nm1847822",2006,2
114,114,6303,False,66000000,"['Fantasy', 'Action']",1487,tt0167190,en,Hellboy,"In the final days of World War II, the Nazis a...",...,122,"['ru', 'en']",Released,Hellboy,6.700,6218,nm0868219,"nm0868219,nm1486009,nm0586005",2004,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10941,12683,751210,False,0,"['Comedy', 'Drama']",96041,tt0217132,fr,La Vie moderne,The lives of three people faced with an uncert...,...,123,['fr'],Released,Modern Life,5.900,6,nm0274262,"nm0274262,nm0346667,nm0449621",2000,2
10953,12695,753303,False,0,['Drama'],110969,tt1922733,fr,Sur la planche,"Set between a numbing, malodorous factory and ...",...,105,"['ar', 'fr']",Released,On the Edge,5.400,5,nm1732967,nm1732967,2011,4
10962,12709,755554,False,0,"['Romance', 'Drama', 'Action']",127896,tt2334114,es,Polvo,Ignacio is shooting a documentary on the peopl...,...,80,['es'],Released,Dust,6.000,1,nm2785946,"nm2785946,nm0409942",2012,2
10965,12712,755948,False,0,['Family'],130370,tt1810520,fr,À pas de loup,In a world and in a time not so faraway lived ...,...,77,['fr'],Released,On the Sly,5.300,8,nm0727599,"nm0727599,nm2070834",2011,2


In [38]:
df[(df['count']== 2) & (df['year']>2020)]['title'].value_counts()

,count
title,
Promises,2
Beast,2
A Brighter Tomorrow,2
The Park,1
No Man's Land,1
Lakshya,1
Enemy,1
Rise,1
Prey,1


# 2. Importer ce qu'il faut pour ne pas afficher les avertissements

In [ ]:
#VA PERMETTRE DE SUPPRIMER LES AVERTISSEMENTS
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# 3. Savoir ce qu'on veut prédire, et à partir de quoi  :
On veut faire un clustering  / KNN: déterminer les 5 films les plus proches du film saisi par l'utilisateur



##  - Supprimer les colonnes qu'on ne veut pas (dans l'entrainement)
Colonnes à garder :
* adult'
* genres'
* original_language'
* popularity'
* production_countries'
* runtime'
* spoken_languages'
* vote_average'
* year' : attention à traiter comme une catégorie

Colonnes à suppprimer :
* budget'
* id_tmdb'
* imdb_id'
* original_title'
* overview'
* poster_path'
* release_date'
* revenue'
* status'
* title'
* vote_count'
* directors'
* writers'

##  - Nettoyer les colonnes (NaN, doublons, format ...)
Pas de NaN dans films  
pas de doublons  

# 3. Déclarer X et y et faire un `.reset_index(drop=True)` sur les 2


**Cible y**  = un titre de film  
**Variables explicatives X** =
* adult'
* genres'
* original_language'
* popularity'
* production_countries'
* runtime'
* spoken_languages'
* vote_average'
* year' : attention à traiter comme une catégorie





# 4. Séparer les variables numériques des variables catégorielles

# 5. Normalisation des valeurs numériques car clustering / KNN


In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_a_normaliser)

print("Données normalisées :")
print(pd.DataFrame(X_scaled, columns=df_a_normaliser.columns, index=df_a_normaliser.index))


# 6. Traitement des valeurs catégorielles / textes (transformation en nombre)
  - get_dummies en priorité (sinon factorize, mais à oublier)

# FONCTION encodage_X(X) qui fait les étapes 3-4-5-6 d'un coup
### Attention, ne pas oublier de Déclarer `X_encoded, SN = encodage_X(X)`

In [ ]:
def encodage_X(X, type='standard'):
  index = X.index
  X_num = X.select_dtypes('number')
  X_cat = X.select_dtypes(['object', 'category', 'string'])
  # Comme ça les dates, on n'y touche pas

  if type == 'standard':
    from sklearn.preprocessing import StandardScaler
    SN = StandardScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  else:
    from sklearn.preprocessing import MinMaxScaler
    SN = MinMaxScaler()
    X_num_SN = pd.DataFrame(SN.fit_transform(X_num), columns=X_num.columns, index=index)

  X_cat_dummies = pd.get_dummies(X_cat)
  X_encoded = pd.concat([X_num_SN, X_cat_dummies], axis=1)

  return X_encoded, SN

In [ ]:
X_encoded, SN = encodage_X(X)

# 7.Grid Search
modèle = KNN
Paramètres


*   n-neighbors
*   metric = euclidien, manhattan, chebyshev


Remarque : pas besoin de faire une validation croisée après le gridsearch car si on utilise le paramètre cv dans le gridsearch, la validation croisée est inclue



In [ ]:
# Définir les modèles à utiliser
models = [
    KNeighborsClassifier(),  # Modèle KNeighborsClassifier
    ]


In [ ]:
# Définir les grilles de recherche pour chaque modèle
param_grids = [
    {'n_neighbors': [3, 5, 7, 9],'metric'=['euclidean', 'manhattan', 'chebyshev']},  # Grille de recherche pour KNeighborsClassifier. On va tester des valeurs de voisins de 3, 5, 7 et 9

]

In [ ]:
# Création et exécution d'un GridSearchCV pour chaque modèle dans une boucle
# On va parcourir chaque model de la liste de modele et y associer chaque valeur distinct des param_grids
for model, param_grid in zip(models, param_grids):
    gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
    gs.fit(X, y)  # Entraînement du modèle sur les données d'entraînement (X, y)

    # Pour chaque modele on affiche le meilleur score obtenu ainsi que les paramètres qui donnent ce score
    print(f"Meilleur score pour {model.__class__.__name__}: {gs.best_score_:.3f}")  # Afficher le meilleur score
    print(f"Meilleurs paramètres pour {model.__class__.__name__}: {gs.best_params_}")  # Afficher les meilleurs paramètres

# 8. Choix et déclaration du modèle
Mettre les paramètres ayant le meilleur score lors du GridSearch

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Création et ajustement du modèle NearestNeighbors
nn = NearestNeighbors(n_neighbors=3, metric='euclidean')
nn.fit(X_scaled)





# 9. Récupération du titre de la cible

Question comment fait-on avec les doublons. Des films différents peuvent avoir le même titre

# 10. Recherche des films similaires à une cible

In [ ]:

# Recherche des fruits similaires à une pomme
pomme_index = df_fruits[df_fruits['nom'] == 'Pomme'].index[0]
distances, indices = nn.kneighbors(X_scaled[pomme_index].reshape(1, -1))

print("\nFruits les plus similaires à une pomme:")
for distance, index in zip(distances[0][1:], indices[0][1:]):  # On exclut la pomme elle-même
    print(f"{df_fruits['nom'].iloc[index]} (Distance: {distance:.2f})")
